# Comparação de Algoritmos de Aprendizado Supervisionado em Problemas de Classificação

- Gabriel Peres de Souza
- João Carlos dos Santos Correia
- Rodrigo Macedo Júnior

## Importando bibliotecas

In [ ]:
#     Importação das bibliotecas.

## Carregamento dos Dados

In [ ]:
#    Carregamento e descrição da base de dados.

## Pré-processamento dos dados (se precisar)

In [ ]:
# dados faltantes, transformação de dados categóricos em numéricos,
# escalas diferentes, outliers, dados desbalanceados, dados duplicados ou inconsistentes, ...

## Implementação dos algoritmos escolhidos

In [ ]:
# Descrição dos Algoritmos

## 1º Algoritmo

In [ ]:
# Implementação

## 2º Algorimo

In [ ]:
# Implementação

# Avaliação dos Modelos com métricas e grafos

In [ ]:
# Lembre-se de utilizar validação cruzada para divisão
# do conjunto de dados e métricas adequadas ao problema de classificação.

# Discussão dos resultados

In [ ]:
# Discussão

## Metodologia
 
Nesta seção, detalhamos o processo adotado para comparar algoritmos de classificação supervisionada, com foco em Regressão Logística (SGD) e k-Vizinhos Mais Próximos (k-NN). O objetivo é avaliar o desempenho dos modelos em um conjunto de dados real, utilizando validação rigorosa e métricas variadas.
 
**Algoritmos utilizados:**
- **Regressão Logística (SGDClassifier, loss='log_loss')**: Algoritmo linear, eficiente para grandes volumes de dados e robusto a ruídos. Utiliza gradiente descendente estocástico para otimização.
- **k-Vizinhos Mais Próximos (k-NN)**: Algoritmo não paramétrico que considera a proximidade entre exemplos para realizar a classificação. Foram testados diferentes valores de *k* e métricas de distância para encontrar a melhor configuração.
 
**Engenharia de atributos:**
A partir das variáveis originais `Date` e `Time`, foram extraídos diversos atributos temporais: ano, mês, dia, dia da semana, dia do ano, hora, minuto, minutos desde meia-noite e indicador de fim de semana. O atributo `Count` foi mantido, e o alvo de classificação é o evento `Event` (binário: 0/1).
 
**Validação e métricas:**
Para garantir uma avaliação robusta, foi utilizada **validação cruzada k-fold (k=10)**, permitindo estimativas mais confiáveis do desempenho dos modelos. As métricas reportadas incluem:
- **Acurácia**: Proporção de acertos totais.
- **Precisão**: Proporção de positivos previstos que são realmente positivos.
- **Sensibilidade (Recall)**: Proporção de positivos reais corretamente identificados.
- **F1-score**: Média harmônica entre precisão e recall.
- **ROC e AUC**: Curva ROC e área sob a curva, avaliando a capacidade de separação dos modelos.
 
## Dados TODO Peres
 
A descrição detalhada dos dados será incluída posteriormente.

## Execução dos scripts
 
- `sgd_logreg.py`: executa a análise com regressão logística (SGD)
- `knn_model.py`: executa a análise com KNN (grid reduzido)
- `test_models.py`: executa ambos e salva os resultados em JSON
- `compare_plots.py`: gera gráficos comparativos (barras e ROC)
 
Os dados gerados ficam em `data/`.

In [ ]:
# Executa os scripts principais para treinar e avaliar os modelos
%run python data/test_models.py
%run python data/compare_plots.py

In [ ]:
import json, pandas as pd
from IPython.display import Image, display
with open('data/sgd_results.json') as f: sgd=json.load(f)
with open('data/knn_results.json') as f: knn=json.load(f)
metrics=['accuracy','precision','recall','f1','roc_auc']
df=pd.DataFrame({'SGD (Regressão logística)':[sgd['metrics_point_on_full'][m] for m in metrics], 'KNN':[knn['metrics_point_on_full'][m] for m in metrics]}, index=metrics)
display(df.style.format('{:.4f}'))
display(Image(filename='data/metrics_comparison.png'))
display(Image(filename='data/roc_both.png'))

In [ ]:
# Carrega e exibe a comparação dos resultados dos modelos
import json, pandas as pd
from IPython.display import Image, display
with open('data/sgd_results.json') as f: sgd=json.load(f)
with open('data/knn_results.json') as f: knn=json.load(f)
metrics=['accuracy','precision','recall','f1','roc_auc']
df=pd.DataFrame({'SGD (Regressão logística)':[sgd['metrics_point_on_full'][m] for m in metrics], 'KNN':[knn['metrics_point_on_full'][m] for m in metrics]}, index=metrics)
display(df.style.format('{:.4f}'))
display(Image(filename='data/metrics_comparison.png'))
display(Image(filename='data/roc_both.png'))